In [1]:
from pathlib import Path
from nilearn import datasets, image, plotting, masking, maskers
from bids import BIDSLayout
import json
from io import StringIO
import numpy as np
import pandas as pd
from scipy import stats

# from niworkflows.interfaces.fixes import FixHeaderApplyTransforms as ApplyTransforms
# from niworkflows.interfaces.itk import MultiApplyTransforms
# from nilearn.connectome import ConnectivityMeasure
# from sklearn.base import clone, BaseEstimator, TransformerMixin
# from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectKBest
# from sklearn.decomposition import TruncatedSVD
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import FunctionTransformer
# import sklearn.datasets
# from sklearn import svm
# from sklearn.metrics import accuracy_score
# from scipy.special import btdtr
# from statsmodels.stats.multitest import multipletests
# import seaborn as sns
# from matplotlib import pyplot as plt
# import itertools
# import dask_ml.model_selection
# import sklearn.model_selection

from joblib import Memory
import os
from joblib import Parallel, delayed
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)


In [2]:
def df_list_merge(df, ll, col_name):
    dfs = []
    for lx in ll:
        tmp = df.copy()
        tmp[col_name] = lx
        dfs.append(tmp)
    return pd.concat(dfs).reset_index(drop=True)

def select_confounds(cfds_path, cfds_sel):
    cfd = pd.read_csv(cfds_path, sep='\t')
    cols = []
    #cfd = cfd.drop('censored', axis=1)
    if "-censor" in cfds_sel: cols.extend([cc for cc in cfd.columns if 'censor' in cc])
    if "-cosine" in cfds_sel: cols.extend([cc for cc in cfd.columns if 'cosine' in cc])
    if "-aroma" in cfds_sel: cols.extend([cc for cc in cfd.columns if 'aroma' in cc])
    if "-motion" in cfds_sel: cols.extend([cc for cc in cfd.columns if ('trans' in cc) or ('rot' in cc)])
    if "-physio" in cfds_sel: cols.extend([cc for cc in cfd.columns if 'phys' in cc])
    cfds = cfd.loc[:, cols].copy()
    return cfds.values

def make_path(ents, updates, pattern, derivatives_dir, build_path, check_exist=True, derivatives=False,
              check_parent=True, mkdir=False, make_parent=False):
    mp_ents = ents.copy()
    if derivatives:
        mp_dir = fmriprep_out / f'sub-{ents["subject"]}/out'
    else:
        mp_dir = derivatives_dir 
    mp_ents.update(updates)
    mp_file = mp_dir / build_path(mp_ents, pattern, validate=False, absolute_paths=False)
    if check_exist and not mp_file.exists():
        raise FileNotFoundError(mp_file.as_posix())
    elif check_parent and not mp_file.parent.exists():
         raise FileNotFoundError(mp_file.parent.as_posix())
    if mkdir:
        mp_file.mkdir(parents=True, exist_ok=True)
    elif make_parent:
        mp_file.parent.mkdir(parents=True, exist_ok=True)
    return mp_file

def r_to_p(r, n):
    from scipy.special import btdtr
    """
    Adapted from Scipy Stats: https://github.com/scipy/scipy/blob/38cd478334a07b6e9dfe224a3ff66ad356c53524/scipy/stats/stats.py#L4069-L4070
    """
    r = np.array(r).copy()
    ab = n / 2 - 1
    prob = 2*btdtr(ab, ab, 0.5 * (1 - np.abs(np.float64(r))))
    return prob

def test_r_to_p():
    rng = np.random.default_rng()
    n = 100
    x = rng.normal(size=n)
    y = rng.normal(size=n)
    r, true_p = stats.pearsonr(x,y)
    assert true_p == r_to_p(r, n)
    y = rng.normal(size=n) + x * 0.3
    r, true_p = stats.pearsonr(x,y)
    assert true_p == r_to_p(r, n)
test_r_to_p()

In [3]:
project_root = Path('..')
bids_dir = project_root / 'data/CATD'
derivatives_dir = bids_dir / 'derivatives'
fmriprep_out = derivatives_dir / 'fmriprep' / 'fmriprep_v21.0.0'

nilearn_out = derivatives_dir / 'rest_processed'
nilearn_out.mkdir(exist_ok=True)
jobids = {}

In [5]:
prep_df = pd.read_csv(derivatives_dir / 'summary_tables' / 'prep_df.csv')
rest_df = prep_df.loc[prep_df.task == 'rest'].copy()
rest_paths = pd.read_csv(derivatives_dir / 'summary_tables' / 'rest_sessamplenothresh_prep.csv')
rest_paths['entities'] = rest_paths['entities'].apply(lambda x: json.loads(x.replace("'", '"')))

database_path='./pybids140_public'
%time layout = BIDSLayout(bids_dir, database_path=database_path)

CPU times: user 2.21 ms, sys: 2.18 ms, total: 4.4 ms
Wall time: 4.54 ms


In [6]:
rest_paths['nv_noncensored'] = (rest_paths.nv - 4) * (1-rest_paths.pct_censored)

In [7]:
rest_paths['above_mfq_thresh'] = rest_paths.s_mfq_tot >= 12

In [8]:
path_cols = ['file'] + list(rest_paths.columns[rest_paths.columns.str.contains('path')])

In [9]:
# rearrange paths to be longish
keep_columns_notd = [
    'subject',
    'run',
    'suffix',
    'session',
    'task',
    'space',
    'datatype',
    'extension',
    'has_phys',
    'sex',
    'group',
    'age',
    'scanner',
    'inpatient',
    'acq_date',
    'usable',
    'dropout',
    'nv_noncensored',
    'above_mfq_thresh',
    'entities',
    'file',
    'mask_path',
    'cfd_new_path',
#     'rest_corr_path',
#     'rest_ts_path'
]
rest_notd = rest_paths.loc[:, keep_columns_notd]
rest_notd['processing'] = 'notd'
# rest_notd = rest_notd.rename(columns={'rest_corr_path': 'orig_rest_corr_path'})
# rest_notd = rest_notd.rename(columns={'rest_ts_path': 'orig_rest_ts_path'})

rest_p = rest_notd.copy()

In [10]:
# atlasses = [image.load_img((nilearn_out / 'basc122_with_schaefer400_ldlpfc.nii.gz').as_posix()),
#             image.load_img((nilearn_out / 'Schaefer2018_400Parcels_7Networks_order_FSLMNI152_2mm.nii.gz').as_posix())]
# atlasses = {'Basc122': image.load_img((nilearn_out / 'basc122_with_schaefer400_ldlpfc.nii.gz').as_posix()),
#             'Schaefer400': image.load_img((nilearn_out / 'Schaefer2018_400Parcels_7Networks_order_FSLMNI152_2mm.nii.gz').as_posix())}
atlasses = {'Basc122': (nilearn_out / 'basc122_flat_with_ldlpfc.nii.gz').as_posix(),
            }
rest_pa = df_list_merge(rest_p, atlasses.keys(), 'atlas')
rest_pa['atlas_img'] = rest_pa.atlas.replace(atlasses)
confounds = ["-censor", "-cosine", "-motion", "-physio"]
rest_pa['counfound_array'] = rest_pa.cfd_new_path.apply(lambda x: select_confounds(x, confounds))

In [11]:
rest_pa['desc'] = rest_pa.processing+rest_pa.atlas
rest_pa['pa_ents'] = rest_pa.loc[:, ['subject', 'session', 'task', 'run', 'space', 'suffix', 'datatype', 'desc']].to_dict('records')

In [12]:
ts_run_name = 'ts_extract_allmodels'
cfds_sel=['censor','cosine','motion','physio']
ts_out = nilearn_out / ts_run_name
ts_out.mkdir(exist_ok=True)
ts_pattern = 'sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_task-{task}_run-{run}_space-{space}_desc-{desc}_{suffix}.{extension}'
ts_updates = {'suffix':'timeseries', 'extension':'csv'}
rest_pa['rest_ts_path'] = rest_pa.pa_ents.apply(lambda x: make_path(x,
                                                            ts_updates,
                                                            ts_pattern,
                                                            ts_out,
                                                            layout.build_path,
                                                            check_exist=False,
                                                            check_parent=False,
                                                            make_parent=True
                                                                          ))

In [13]:
def extract_ts_and_save(row, n_dummy=4):
    regions_extracted_img = image.load_img(row.atlas_img)
    img = image.load_img(row.file)
    
    masker = maskers.NiftiLabelsMasker(regions_extracted_img,
                                          mask_img=row.mask_path,
                                          resampling_target="data",
                                          low_pass=.1,
                                          high_pass=.01,
                                          t_r=2.5)
    extracted = masker.fit_transform(row.file, confounds=row.counfound_array)[n_dummy:,:]
    pd.DataFrame(extracted).to_csv(row.rest_ts_path, header=None, index=None)
    return extracted

In [14]:
ts_exist = rest_pa.rest_ts_path.apply(lambda x: x.exists())


In [15]:
extract_jobs = rest_pa.loc[~ts_exist].apply(lambda row: delayed(extract_ts_and_save)(row.copy()), axis=1)
#extract_jobs = rest_pa.apply(lambda row: delayed(extract_ts_and_save)(row.copy()), axis=1)

In [16]:
len(extract_jobs)

504

In [18]:
if len(extract_jobs) > 0:
    res = Parallel(n_jobs=20)(extract_jobs)
extracted = rest_pa.loc[ts_exist, 'rest_ts_path'].apply(lambda x: pd.read_csv(x,header=None).values)

/gpfs/gsfs11/users/MBDU/mid_share/env/lib/python3.9/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning: invalid value encountered in true_divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)
/gpfs/gsfs11/users/MBDU/mid_share/env/lib/python3.9/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning: invalid value encountered in true_divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)
/gpfs/gsfs11/users/MBDU/mid_share/env/lib/python3.9/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning: invalid value encountered in true_divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)
/gpfs/gsfs11/users/MBDU/mid_share/env/lib/python3.9/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning: invalid value encountered in true_divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)
/gpfs/gsfs11/users/MBDU/mid_share/env/lib/python3.9/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning: invalid

In [19]:
# make sure extraction worked
ts_exist = rest_pa.rest_ts_path.apply(lambda x: x.exists())
assert ts_exist.all()

In [20]:
rest_pa['rest_ts'] = extracted

In [21]:
rest_pa.to_csv(derivatives_dir / 'summary_tables' / 'rest_paths_with_timeseries.csv', index=None)